In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import Imputer

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("imputation_data_test.csv")
df

,ID,Gender,Age,PhD_degree,Level,Salary,Location,Happiness
0,101,Male,20.0,0.0,A,100.0,MD,Yes
1,102,NaN,26.0,NaN,A,NaN,VA,No
2,103,Female,30.0,0.0,B,120.0,NaN,No
3,104,Male,28.0,1.0,C,140.0,MD,NaN
4,105,Male,54.0,1.0,A,200.0,MD,Yes
5,106,Male,18.0,1.0,B,NaN,NaN,No
6,107,Male,50.0,1.0,NaN,100.0,DC,Yes
7,108,Female,60.0,NaN,A,115.0,NaN,No
8,109,Male,NaN,1.0,C,100.0,NaN,No
9,110,NaN,20.0,0.0,C,180.0,PA,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
ID            10 non-null int64
Gender        8 non-null object
Age           9 non-null float64
PhD_degree    8 non-null float64
Level         9 non-null object
Salary        8 non-null float64
Location      6 non-null object
Happiness     9 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 720.0+ bytes


In [ ]:
# replace empty by nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df

In [ ]:
df.info()

In [ ]:
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?','np.nan',regex=True)

## Data Cleaning

In [5]:
# Drop the column `ID` as it is unrelated to the dependent variable
df = df.drop(['ID'],axis=1)
df

,Gender,Age,PhD_degree,Level,Salary,Location,Happiness
0,Male,20.0,0.0,A,100.0,MD,Yes
1,NaN,26.0,NaN,A,NaN,VA,No
2,Female,30.0,0.0,B,120.0,NaN,No
3,Male,28.0,1.0,C,140.0,MD,NaN
4,Male,54.0,1.0,A,200.0,MD,Yes
5,Male,18.0,1.0,B,NaN,NaN,No
6,Male,50.0,1.0,NaN,100.0,DC,Yes
7,Female,60.0,NaN,A,115.0,NaN,No
8,Male,NaN,1.0,C,100.0,NaN,No
9,NaN,20.0,0.0,C,180.0,PA,Yes


In [6]:
# Remove rows where taget are missing
missing_row = df['Happiness'].isnull()
print('Number of rows where target are missing:')
print(sum(missing_row))

df = df[~missing_row]
df

Number of rows where target are missing:
1


,Gender,Age,PhD_degree,Level,Salary,Location,Happiness
0,Male,20.0,0.0,A,100.0,MD,Yes
1,NaN,26.0,NaN,A,NaN,VA,No
2,Female,30.0,0.0,B,120.0,NaN,No
4,Male,54.0,1.0,A,200.0,MD,Yes
5,Male,18.0,1.0,B,NaN,NaN,No
6,Male,50.0,1.0,NaN,100.0,DC,Yes
7,Female,60.0,NaN,A,115.0,NaN,No
8,Male,NaN,1.0,C,100.0,NaN,No
9,NaN,20.0,0.0,C,180.0,PA,Yes


We determine and drop the variables with excessive missing values from the dataset.

In [7]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df   

In [8]:
df = remove_bad_columns(df,3)
df.info()

number of bad columns: 1


,Gender,Age,PhD_degree,Level,Salary,Happiness
0,Male,20.0,0.0,A,100.0,Yes
1,NaN,26.0,NaN,A,NaN,No
2,Female,30.0,0.0,B,120.0,No
4,Male,54.0,1.0,A,200.0,Yes
5,Male,18.0,1.0,B,NaN,No
6,Male,50.0,1.0,NaN,100.0,Yes
7,Female,60.0,NaN,A,115.0,No
8,Male,NaN,1.0,C,100.0,No
9,NaN,20.0,0.0,C,180.0,Yes


We find bad rows which contain too many missing values, then remove them.

In [9]:
def remove_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    df = df.drop(bad_row)
    #df.info()
    return df

In [10]:
df = remove_bad_rows(df,3)
df.info()

number of bad rows: 1


,Gender,Age,PhD_degree,Level,Salary,Happiness
0,Male,20.0,0.0,A,100.0,Yes
2,Female,30.0,0.0,B,120.0,No
4,Male,54.0,1.0,A,200.0,Yes
5,Male,18.0,1.0,B,NaN,No
6,Male,50.0,1.0,NaN,100.0,Yes
7,Female,60.0,NaN,A,115.0,No
8,Male,NaN,1.0,C,100.0,No
9,NaN,20.0,0.0,C,180.0,Yes


For convenience, we separate independents `X` and dependent `y` from the data.

In [11]:
dfx = df.drop('Happiness',axis=1)
dfy = df['Happiness']

In [12]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[2 6 2 3 5]


In [13]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [14]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [0, 2]
i_category: [3]
[1. 3. 1. 2. 3.]


In [15]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [16]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed

,Gender,Age,PhD_degree,Level,Salary
0,Male,20.0,0.0,A,100.0
2,Female,30.0,0.0,B,120.0
4,Male,54.0,1.0,A,200.0
5,Male,18.0,1.0,B,115.0
6,Male,50.0,1.0,A,100.0
7,Female,60.0,1.0,A,115.0
8,Male,30.0,1.0,C,100.0
9,Male,20.0,0.0,C,180.0


In [17]:
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 9
Data columns (total 5 columns):
Gender        8 non-null object
Age           8 non-null float64
PhD_degree    8 non-null float64
Level         8 non-null object
Salary        8 non-null float64
dtypes: float64(3), object(2)
memory usage: 384.0+ bytes


## Data Processing

### Attributes

In [18]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [19]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(8, 7)
[[  1.  20.  -1.   1.   0.   0. 100.]
 [ -1.  30.  -1.   0.   1.   0. 120.]
 [  1.  54.   1.   1.   0.   0. 200.]
 [  1.  18.   1.   0.   1.   0. 115.]
 [  1.  50.   1.   1.   0.   0. 100.]
 [ -1.  60.   1.   1.   0.   0. 115.]
 [  1.  30.   1.   0.   0.   1. 100.]
 [  1.  20.  -1.   0.   0.   1. 180.]]


### Target

In [20]:
y = np.array(dfy)
print(np.unique(y,return_counts=True))

# convert target to 0 and 1
y_new = np.ones(y.shape[0])
y_new[y =='No'] = 0

# if target is already 0 and 1
#y_new = y #and delete lines below convert  
print(np.unique(y_new,return_counts=True))

(array(['No', 'Yes'], dtype=object), array([4, 4]))
(array([0., 1.]), array([4, 4]))


In [21]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')